# PROYECTO 2 #
# Ingeniería de features, Modelos avanzados e Interpretación de modelos
## PROYECTO: Análisis de mercado inmobiliario ##
### PROBLEMA ###  
Recientemente te has incorporado al equipo de Datos de una gran inmobiliaria. La primera tarea que se te asigna es ayudar a los tasadores/as a valuar las propiedades, ya que es un proceso difícil y, a veces, subjetivo. Para ello, propones crear un modelo de Machine Learning que, dadas ciertas características de la propiedad, prediga su precio de venta.
### RESUMEN DEL PROYECTO ###
Aplica transformación de datos y entrena Modelos Avanzados para desarrollar con mayor profundidad tu modelo de Machine Learning. ¿Qué puedes aprender del problema que estás abordando mediante el estudio de tu propio modelo?
### CONSIGNA ###
En este proyecto profundizarás lo desarrollado en el proyecto 01 (“Primer modelo de Machine Learning”). El objetivo es aplicar las técnicas incorporadas (Transformación de Datos, Optimización de Hiperparámetros, Modelos Avanzados, etc.) para generar un modelo que tenga un mejor desempeño que el modelo generado en el proyecto anterior. Luego, interpreta ese modelo para responder la siguiente pregunta: ¿qué podemos aprender de nuestro problema estudiando el modelo que generamos?
## PARTE A - Transformación de Datos ###

Elige cuáles de las siguientes tareas son apropiadas para su dataset. Justifica e implementa:

* Detección y eliminación de Outliers

* Encoding

* Imputación de valores faltantes

* Escalado de datos

* Generación de nuevas variables predictoras/reducción de dimensionalidad (SVD/PCA).

Vuelve a entrenar el modelo implementado en la Entrega 01 - en particular, el árbol de decisión - y evalúa su desempeño a partir del dataset obtenido luego de transformar los datos. ¿Hay una mejora en su desempeño? Sea cual sea la respuesta, intenta explicar a qué se debe.

### 1. Análisis Exploratorio de Datos

1. __Se importan las librerías__ necesarias para trabajar en la consigna.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

ImportError: cannot import name 'rcsetup' from partially initialized module 'matplotlib' (most likely due to a circular import) (C:\Users\Ale\anaconda3\envs\ds\lib\site-packages\matplotlib\__init__.py)

2. __Se realiza la carga el dataset__ usando las funcionalidades de Pandas.

In [ ]:
properati = pd.read_csv('DS_Proyecto_01_Datos_Properati.csv')

In [ ]:
properati.shape # Filas y columnas

* *El Dataset, cuenta con **146.660 Filas**, y **19 Columnas**.*

In [ ]:
properati.head(3) # Primeras 3 instancias (filas)

3. __Valores Faltantes:__ se imprimen en pantalla los nombres de las columnas y cuántos valores faltantes hay por columna. En un principio es a mera exposición, ya que por el momento no vamos a descartar ninguno de ellos,ni realizar imputación de datos.

In [ ]:
properati.isnull().sum() # Nombres de las columnas y su cantidad de faltantes

* *Variables con elementos faltantes:*  
    *1. `Superficie Cubierta` **15%** (21.614);*  
    *2. `Superficie Total` **14%** (20.527);*  
    *3. Latitud y Longitud 7% c/u (10.000 c/u);*  
    *4. Baños 4% (6.000).*

4. __Tipos de propiedad:__ Se explora cuántos tipos de propiedad hay publicados en el dataset y la cantidad de instancias por cada tipo de propiedad.

In [ ]:
pd.unique(properati['property_type'])

In [ ]:
print(properati['property_type'].value_counts())

* *En el dataset de Properati se encuentran publicados __10 Tipos de Propiedad__ en la zona geográfica analizada.*  
* *Se destacan:*  
    *1. `Departamento` con 107.326 unidades __(73%)__;*  
    *2. `Casa` con 21.521 (15%);*  
    *3. `PH` con 14.298 instancias (10%).*

In [ ]:
sns.countplot(data = properati, y = 'property_type', order = properati['property_type'].value_counts().index, palette='pastel')
plt.title('Número de Publicaciones por Tipo de Propiedad')

5. __Se visualizan las regiones__ a las cuales pertenecen las publicaciones.

In [ ]:
f, ax = plt.subplots(figsize = (7,2))
sns.countplot(y = 'l2', data = properati, order = properati['l2'].value_counts().index, color = '#82B3FF')
plt.title('Número de Publicaciones por Zona Urbana de la Provincia de Bs. As.')

f, ax = plt.subplots(figsize = (7,5))
sns.countplot(y = 'l3', data = properati, order = properati['l3'].value_counts().iloc[:10].index, color = '#957dad')
plt.title('Número de Publicaciones por Barrio/Partido de la Provincia de Bs. As')

In [ ]:
print(properati['l2'].value_counts())

In [ ]:
print(properati['l3'].value_counts())

* *Las publicaciones son principalmente de `Capital Federal` **(63%)**.*  
* *Dentro de CABA, se detaca el barrio de `Palermo` (**14%** s/ CABA), seguido por Almagro, Belgrano, Caballito, Villa Crespo y Recoleta.*  
* *Dentro de AMBA, el partido de `Tigre` es el que presenta mayor número de publicaciones.*

6. __Se exponen las Estadísticas Descriptivas__, como ser la tendencia central, la dispersión y la forma de la distribución de un conjunto de datos, excluyendo los NaN valores.

In [ ]:
pd.set_option('precision', 2) # Para una mejor visualización, se redujeron los decimales a 2

In [ ]:
properati[['rooms','bedrooms','bathrooms','surface_total','surface_covered','price']].describe().round()

* *`Surface Total` y `Surface Covered`: los valores mínimos y máximos obtenidos resultan poco razonables para dichas instancias, ya que como se observa, oscilan entre 10m2 - 193.549m2 y 1m2 - 126.062m2 respectivamente. Al respecto, se observa una `Desviación Estándar Alta`, lo cual indica que los datos se extienden sobre un amplio rango de valores.*
* *`Bedrooms = 0`, es coherente, ya que podría tratarse de `Monoambientes`, donde se comparten en un mismo ambiente, living, cocina y dormitorio.*
* *`Bathrooms = 0`, es lógica por estar trabajando con Depósitos y Lotes por ejemplo, que pueden no tener baños.*
* *`Price`: Los mínimos y máximos distan mucho de la media.*

## 2. Transformación de Datos

### 2.1 Detección y Eliminación de Outliers

1. __Detección de Outliers__ a través de Boxplots, de las variables `Precio`, `Superficie Total` y `Superficie Cubierta` por considerarlas más relevantes y con mayor dispersión de datos s/ las estadísticas descriptivas previamente expuestas.

In [ ]:
plt.figure(figsize = (12,6))
sns.boxplot(data = properati, x = 'property_type', y = 'price')
plt.title('Precio en Dólares por Tipo de Propiedad')
plt.ticklabel_format(axis = 'y', style = 'plain')

In [ ]:
plt.figure(figsize = (12,6))
sns.boxplot(data = properati, x = 'property_type', y = 'surface_total')
plt.title('Superficie Total en m2 por Tipo de Propiedad')

In [ ]:
plt.figure(figsize = (12,6))
sns.boxplot(data = properati, x = 'property_type', y = 'surface_covered')
plt.title('Superficie Cubierta en m2 por Tipo de Propiedad')

* *Tanto en `Precio` como en `Superficie Total` se visualizan una gran cantidad de outliers.*
* *En el caso de `Precio`, los outliers más distantes de la media (por encima de ella) se observan en `Departamento` y `Otro`.*
* *En el caso de `Superficie Total`, los outliers más distantes de la media se observan en `Departamento`, `Lote` y `Otro`.*
* *`Superficie Cubierta` presenta outliers principalmente en `Departamento`, `Casa` y `PH`.*
* *También se observan valores **Nulos**, lo cual no es consistente.*

2. __Eliminación de Outliers:__ se procede a descartar los datos atípicos para cada tipo de propiedad.
* *Primero, se realiza un **primer filtro** en el dataset, aplicando **IQR Score**, para la variable `Precio`.*

In [ ]:
Q1_p = properati['price'].quantile(0.25)
Q3_p = properati['price'].quantile(0.75)
IQR_p = Q3_p - Q1_p
print (IQR_p) # Se visualiza el IQR para columna precios

In [ ]:
mask_price = properati [~ ((properati['price'] <(Q1_p - 1.5 * IQR_p)) | (properati['price']> (Q3_p + 1.5 * IQR_p)))]
properati_2 = mask_price

properati_2['price'].describe().round()

In [ ]:
properati_2.shape

* *Un **7,5%** de los datos resultaron ser atípicos, de acuerdo a la metodología utilizada (IQR Score).*

* *Luego, se procede a filtrar aquellas instancias en las que la `Superficie Cubierta` sea superior a la `Superficie Total`, ya que en la práctica NO es posible que ésto suceda.*

In [ ]:
mask_sup = (properati_2['surface_covered'] <= properati_2['surface_total'])
properati_3 = properati_2[mask_sup]
properati_3[['price','surface_total']].describe().round()

In [ ]:
properati_3.shape

* *La muestra se redujo casi un **15%** más.*

* *__Análisis:__ se procede a visualizar la nueva distribución de los datos a partir del primer filtro realizado.*

In [ ]:
properati_3[['rooms','bedrooms','bathrooms','surface_total','surface_covered','price']].describe().round()

In [ ]:
plt.figure(figsize = (12,6))
sns.boxplot(data = properati_3, x = 'property_type', y = 'price', palette= 'pastel')
plt.title('Precio en Dólares por Tipo de Propiedad')
plt.ticklabel_format(axis = 'y', style = 'plain')

In [ ]:
plt.figure(figsize = (12,6))
sns.boxplot(data = properati_3, x = 'property_type', y = 'surface_total')
plt.title('Superficie Total en m2 por Tipo de Propiedad')

In [ ]:
plt.figure(figsize = (12,6))
sns.boxplot(data = properati_3, x = 'property_type', y = 'surface_covered')
plt.title('Superficie Cubierta en m2 por Tipo de Propiedad')

* *La distribución de datos de `Precio` expone mayor robustez.*
* *Sin embargo, en los casos de `Superficie Total` y `Superficie Cubierta` los valores atípicos siguien distando mucho de la media, oscilando entre 10m2 - 169.000m2 y 1m2 - 126.062m2 respectivamente. La Desviación Estándar sigue siendo Elevada.*

* *Se realiza un **segundo filtrado**, con el fin de alcanzar valores razonables en las `Superficies`. Comenzamos con `Superficie Total`.*

In [ ]:
Q1_st = properati_3['surface_total'].quantile(0.25)
Q3_st = properati_3['surface_total'].quantile(0.75)
IQR_st = Q3_st - Q1_st
print (IQR_st) # Se visualiza el IQR para columna precios

In [ ]:
mask_st = properati_3 [~ ((properati_3['surface_total'] <(Q1_st - 1.5 * IQR_st)) | (properati_3['surface_total']> (Q3_st + 1.5 * IQR_st)))]
properati_4 = mask_st

properati_4['surface_total'].describe().round()

In [ ]:
properati_4.shape

* *En ésta ocación, la muesta fue acotada un **7,5% más**, en comparación al dataset original.*

* *__Nuevo Análisis:__ a fin de observar la robustez de los datos.*

In [ ]:
properati_4[['rooms','bedrooms','bathrooms','surface_total','surface_covered','price']].describe().round()

In [ ]:
plt.figure(figsize = (12,6))
sns.boxplot(data = properati_4, x = 'property_type', y = 'price', palette= 'pastel')
plt.title('Precio en Dólares por Tipo de Propiedad')
plt.ticklabel_format(axis = 'y', style = 'plain')

In [ ]:
plt.figure(figsize = (12,6))
sns.boxplot(data = properati_4, x = 'property_type', y = 'surface_total', palette= 'pastel')
plt.title('Superficie Total en m2 por Tipo de Propiedad')

In [ ]:
plt.figure(figsize = (12,6))
sns.boxplot(data = properati_4, x = 'property_type', y = 'surface_covered', palette= 'pastel')
plt.title('Superficie Cubierta en m2 por Tipo de Propiedad')

* *Ahora sí. Tanto la distribución de datos de `Precio` como de las `Superficies` exponen mayor robustez.*
* *Si bien aún se verifican algunos Outliers principalmente en precios, por cantidad y valores que alcanzan, sólo lo serían por tipo de propiedad y no considerando el conjunto total de los datos.*

### 2.2 Imputación de Valores Faltantes

In [ ]:
properati_4.isnull().sum()

* *Variables que aún cuentan con elementos faltantes:*  
    *1. Latitud y Longitud 6,5% c/u (6.621 c/u);*  
    *2. Baños 1,3% (1.345).*

* *Nos enfocaremos en la imputación de datos de `Baños`, ya que en el próximo paso, trabajaremos con Departamento, Casa y PH (a fin de comparar nuestros resultados con los del Protyecto 1), y no es posible que no hayan Baños en dichos tipos de propiedades (como sí podría ocurrir en un Depósito).*
* *Respecto a latitud y longitud, no se realizará ningún cambio, ya que no serán utilizadas en éste estudio para determinar el precio de las propiedades por la complejidad de su análisis.*

In [ ]:
plt.figure(figsize = (7,5))
sns.distplot(properati_4['bathrooms']) 
plt.title('Distribución de la variable Baños')

In [ ]:
median_bathrooms = properati_4['bathrooms'].median()
print(median_bathrooms)

In [ ]:
mode_bathrooms = properati_4['bathrooms'].mode()
print(median_bathrooms)

In [ ]:
mean_bathrooms = properati_4['bathrooms'].mean() # Ya espuesto con anterioridad
print(median_bathrooms)

* *Si bien alguna de las propiedades posee 14 baños, dicho valor atípico no está influyendo en las medidas de tendencia central, ya que la media, la moda y la mediana son igual a **1.0**. A continuación, **se procederá a imputar los datos faltantes con dicho valor**.*
* *Se agrega que la distribución de sus datos es asímétrica a la derecha (positiva), y la cantidad de valores nulos representa un porcentaje bajo sobre el total. Ésto ocurre por la cantidad de valores nulos observados; sin ellos y sin el inmueble con 14 baños, y dados los valores de la media, mediana y moda (iguales entre sí), sería razonable obserbar una distribución normal de los datos en baños.*

In [ ]:
properati_4['bathrooms'] = properati_4['bathrooms'].fillna(median_bathrooms)

In [ ]:
sum(pd.isnull(properati_4['property_type']))

* *Todos los valores nulos de `Baños` fueron reemplazados por **1.0**, es decir por el valor de la __media, mediana y moda__.*

### 2.3 Selección de la Muestra

* *A fin de **comparar los resultados** de los modelos del Proyecto 2 **con los resultantes del Proyecto 01**, utilizaremos la misma muestra, es decir, que nos centraremos en:*
    * *Los tipos de propiedad con mayor concentración, `Departamento, Casa y PH` y;*
    * *En la región con mayor número de publicaciones, es decir, `Capital Federal`.*

In [ ]:
properati_5 = properati_4 [(properati_4['l2'] == 'Capital Federal') & ((properati_4['property_type'] == 'Departamento') | (properati_4['property_type'] == 'PH') | (properati_4['property_type'] == 'Casa'))]

In [ ]:
properati_5.shape

* *La muestra se redujo en un **8% más**.*

* *Se exponen nuevamente las Estadísticas Descriptivas, con el Dataset Filtrado.*
* *En su revisión, se verifica que se cumpla que la **Superficie Cubierta Mínima sea de 18m2**, basándose en el Nuevo Código de Edificación de CABA, que entró en vigencia el 1° de enero de 2019, y hablitó la construcción de **microambientes mínimos**, de hasta 18 metros cuadrados. Se puede verificar en [Nuevo Código de Edificación](https://www.buenosaires.gob.ar/desarrollourbano/nuevo-codigo-de-edificacion).*

In [ ]:
properati_5[['rooms','bedrooms','bathrooms','surface_total','surface_covered','price']].describe()

* *Mejora la distribución de los datos, presentando una reducción notable en el desvío estándar de los mismos.*
* *No se verifica el cumplimiento de la `Superficie Cubierta Mínima` de 18m2, por lo que a continuación, lo hacemos cumplir:*

In [ ]:
mask_sup2 = (properati_5['surface_covered'] >= 18)
properati_6 = properati_5[mask_sup2]

properati_6['surface_covered'].describe().round()

In [ ]:
properati_6.shape

* *El **Dataset Final con el que vamos a trabajar**, representa aprox. un **49% del Dataset Original**.*
* *Si bien puede parecer un porcentaje bajo, eran muchos los outliers (como propiedades con Sup. Total de 126.062m2) o mal cargados (como Sup. Total < que Sup. Cubierta).*

* *A continuación se reflejan las **nuevas distribuciones** para las `variables Superficie Total` y `Precio`:*

In [ ]:
plt.figure(figsize = (6,5))
sns.distplot(properati_6['price'])
plt.title('Distribución de la variable Precio')
plt.ticklabel_format(style = 'plain')

In [ ]:
plt.figure(figsize = (6,5))
sns.boxplot(data = properati_6, x = 'property_type', y = 'price', palette= 'pastel')
plt.title('Precio en Dólares por Tipo de Propiedad')

In [ ]:
plt.figure(figsize = (6,5))
sns.distplot(properati_6['surface_total']) 
plt.title('Distribución de la variable Superficie Total')

In [ ]:
plt.figure(figsize = (6,5))
sns.boxplot(data = properati_6, x = 'property_type', y = 'surface_total', palette= 'pastel')
plt.title('Superficie Total en m2 por Tipo de Propiedad')

In [ ]:
plt.figure(figsize = (6,5))
sns.distplot(properati_6['surface_covered']) 
plt.title('Distribución de la variable Superficie Cubierta')

In [ ]:
plt.figure(figsize = (6,5))
sns.boxplot(data = properati_6, x = 'property_type', y = 'surface_covered', palette= 'pastel')
plt.title('Superficie Cubierta en m2 por Tipo de Propiedad')

* *Tanto en el caso de Superficie como de Precios, se visualizan datos más consistentes.*
* *En ambos casos, es clara la distribución de datos con **asimetría positiva (o a la derecha)**, siendo la mayor parte de los precios de las propiedades, menores a U$S 250.000.*

In [ ]:
#sns.pairplot(data=properati_6, hue= 'property_type', vars=['rooms','bedrooms','bathrooms','surface_total','surface_covered','price'])

* *Podemos inferir que tanto Superficie Total como Superficie Cubierta, están altamente correlacionadas con el Precio, no así el resto de las variables.*

### 2.4 Correlaciones 

1. __Correlaciones Pearson:__ Primero se realiza el estudio de las correlaciones entre las variables `rooms, bedrooms, bathrooms, surface_total, surface_covered` y `price`, con el fin de exponer la existencia de correlaciones lineales.

In [ ]:
corr = properati_6[['rooms','bedrooms','bathrooms','surface_total','surface_covered','price']].corr()
plt.figure(figsize=(6,6))
sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 15}, cmap= 'Set2')
plt.xticks(rotation = 45)
plt.yticks(rotation = 45)
plt.title('Correlación entre Variables - Pearson')
plt.show()

* *Se observa:*
    * *Correlación Alta **(0.93)**, entre `rooms` (ambientes) y `bedrooms` (dormitorios).*
    * *Correlación Alta **(0.92)**, entre `surface_covered` (superficie total) y `surface_total` (superficie cubierta).*
    * *Además, `surface_covered` y `surface_total`, tiene Correlación Alta y Moderadamente Alta, con todas las demás variables, excepto `bathrooms`.*
* *Por su parte, `Price` posee una correlación Moderadamente Alta con `surface_covered` **(0.75)** y `surface_total` **(0.73)**, coincidiendo con lo esperado, respecto a ser las __variables más relevantes para determinar el precio de los inmuebles__.*

2. __Correlaciones Spearman:__ Para adicionar información al estudio, se expone la correlación a través del método Spearman, con el fin de visualizar la existencia de correlaciones No lineales.

In [ ]:
corr = properati_6[['rooms','bedrooms','bathrooms','surface_total','surface_covered','price']].corr(method='spearman')
plt.figure(figsize=(6,6))
sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 15}, cmap= 'Set2')
plt.xticks(rotation = 45)
plt.yticks(rotation = 45)
plt.title('Correlación entre Variables - Spearman')
plt.show()

* *No se observan importantes cambios entre ambos tipos de correlaciones.*
* *Se mantienen las Altas correlaciones entre `rooms` (ambientes) y `bedrooms` (dormitorios), y entre `surface_covered` (superficie total) y `surface_total` (superficie cubierta).*
* `surface_covered` y `surface_total`, incrementan su correlación positiva con`rooms` y `bedrooms`*.
* *Por su parte, `Price` (precio) posee una correlación algo mayor con `surface_covered` **(0.79)** y `surface_total` **(0.80)**.*

### 2.5 Escalado de Datos

* *Primero, se analiza si algunas de las variables requiere escalado.*

In [ ]:
sns.jointplot(x='surface_total', y='price', data=properati_6, color = '#82B3FF')

In [ ]:
sns.jointplot(x='surface_covered', y='price', data=properati_6, color = '#82B3FF')

In [ ]:
sns.jointplot(x='rooms', y='price', data=properati_6, color = '#82B3FF')

In [ ]:
sns.jointplot(x='bedrooms', y='price', data=properati_6, color = '#82B3FF')

In [ ]:
sns.jointplot(x='bathrooms', y='price', data=properati_6, color = '#82B3FF')

* *En `Rooms`, `Bedrooms` y `Bathrooms` parece preciso aplicar escalado a sus datos, por observarse en ellos, valores atípicos muy grandes que pueden degradar el rendimiento predictivo de los algoritmos de aprendizaje automático.*
* *Igualmente, el escalado de datos, **se realizará para las 5 características** previamente expuestas en los gráficos y para `Precio`, con el fin de **normalizar sus datos**, dentro de un rango particular.*
* *Se procede a realizar dicho proceso a través de **RobustScaler**, ya que, a diferencia de StandardScaler, sus estadísticas de centrado y escalado, se basan en percentiles y, por lo tanto, no están influenciadas por unos pocos valores atípicos marginales muy grandes (como en Baños). En consecuencia, el rango resultante de los valores de las características transformadas es mayor que para StandardScaler y son aproximadamente similares.*

In [ ]:
from sklearn.preprocessing import RobustScaler

properati_scaler = RobustScaler().fit_transform(properati_6[['rooms','bedrooms','bathrooms','surface_total','surface_covered','price']])

properati_scaler.shape

In [ ]:
properati_7 = pd.DataFrame(properati_scaler, index = properati_6[['rooms','bedrooms','bathrooms','surface_total','surface_covered','price']].index,
                         columns = properati_6[['rooms','bedrooms','bathrooms','surface_total','surface_covered','price']].columns)

* *Los datos que utilizaremos en los siguientes modelos, han sido escalados.*

In [ ]:
sns.jointplot(x='bathrooms', y='price', data=properati_7, color = '#82B3FF')

* *Teniendo como ejemplo a Baños, la distribución es similar.*
* *Sin embargo, los datos de todas las variables elegidas, han sido normalizados tal cual esperábamos.*

In [ ]:
properati_7.head()

* *El **Dataset Escalado**, incluye únicamente las columnas de `rooms, bedrooms, bathrooms, surface_total, surface_covered` y `price`.*

### 2.6 Aplicación de Reducción de Dimensionalidad - PCA

* *Se implementa la técnica de PCA, ya que es particularmente útil en el tratamiento de datos donde existen múltiples - colinealidades entre las características / variables, como ocurre en el presente estudio.*
* *El análisis de componentes principales es una técnica matemática utilizada para la reducción de dimensionalidad. Su objetivo es reducir el número de features, conservando la mayor parte de la información original.

* *Como **X** vamos a considerar las 5 variables escaladas que sirven como predictoras, con el fin de luego reducir su dimensionalidad y como **y** al Precio.*
* *Como se expuso anteriormente, entre ellas existen múltiples correlaciones, y a su vez, algunas de ellas parece tener influencia en la determinación de los precios (como las superficies), y otras no (ambientes, dormitorios y baños).*

1. Se **seleccionan las variables** predictoras (`X`) y la variable a predecir (`y`).

In [ ]:
X = properati_7[['surface_total','surface_total','bedrooms','rooms','bathrooms']]
y = properati_7[['price']]

In [ ]:
X[:5]

2. Se define la **matriz de Covarianza**.

In [ ]:
# Matriz de covarianza
features = X.T
cov_matrix = np.cov(features)
cov_matrix[:5]

* En la diagonal de la matriz de covarianzas, tenemos varianzas, y los demás elementos son las covarianzas.
* Los elementos diagonales son idénticos y la matriz es simétrica.

3. Se realiza la **Eigendecomposition**.

In [ ]:
# Eigendecomposition
valores, vectores = np.linalg.eig (cov_matrix) 
valores [: 5]

In [ ]:
vectores [: 5]

* *A partir de esto, podemos calcular el porcentaje de varianza explicada (explained variance) por componente principal:*

In [ ]:
varianzas_explicadas = [] 
for i in range (len (valores)): 
    varianzas_explicadas.append (valores [i] / np.sum (valores)) 
 
    print (np.sum (varianzas_explicadas), '\n', varianzas_explicadas)

* *El primer valor (penúltima fila) es solo la suma de las varianzas explicadas y debe ser igual a 1. El segundo valor (última fila) es una matriz, que representa el porcentaje de varianza explicada por componente principal.*
* *El primer componente principal representa el 81% de la varianza de los datos, el segundo el 11%.*

* 4. **Visualizaciones**.

In [ ]:
proyectado_1 = X.dot (vectores.T [0]) 
proyectado_2 = X.dot (vectores.T [1])
res = pd.DataFrame (proyectado_1, columns = ['PC1']) 
res ['PC2'] = proyectado_2 
res ['Y'] = y 
res.head ()

* *Primero se visualiza el conjunto de datos en una dimensión: como una línea (no incluímos a PC2).*

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 5))
sns.scatterplot (res ['PC1'], [0] * len (res), hue = res ['Y'], s = 200)

* *Luego, se exponen los datos en un espacio 2D:*

In [ ]:
plt.figure (figsize = (10, 5)) 
sns.scatterplot(res['PC1'], res['PC2'], hue=res['Y'], s=100)

* *En ambos gráficos se observa que las variables son difíciles de separar.*

5. Vemos cómo funciona el **modelo PCA**, con 2 componentes principales.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
print("original shape:   ", X.shape)
print("transformed shape:", X_pca.shape)

In [ ]:
print(pca.components_)

* *Los datos transformados se han reducido a una sola dimensión.*
* *La reducción de dimensionalidad de PCA, eliminó la información a lo largo del eje o ejes principales menos importantes, dejando solo el componente o componentes de los datos con la mayor varianza, en éste caso, el primer y segundo componente.*
* *Este conjunto de datos de dimensión reducida es, en algunos sentidos, "lo suficientemente bueno" para codificar las relaciones más importantes entre los puntos: a pesar de reducir la dimensión de los datos, la relación general entre los puntos de datos se conserva en su mayoría.*

## 3. Entrenamiento del modelo implementado en la Entrega 01 - Árbol de Decisión

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.30, random_state=10)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

lista_rmse_train_dt = []
lista_rmse_test_dt = []

max_depths = [1,2,3,4,5,6,7,8,9,10,20,25,30,40,50,80]

for max_depth in max_depths:

    # Se define el modelo con la profundidad deseada
    tree_regressor = DecisionTreeRegressor(max_depth = max_depth, random_state=10)
    
    tree_regressor.fit(X_train, y_train)
    
    y_train_pred = tree_regressor.predict(X_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    
    y_test_pred = tree_regressor.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    lista_rmse_train_dt.append(rmse_train)
    lista_rmse_test_dt.append(rmse_test)

In [ ]:
plt.figure(figsize = (8,4))
plt.plot(max_depths, lista_rmse_train_dt,'o-',label='train' )
plt.plot(max_depths, lista_rmse_test_dt,'o-',label='test')
plt.legend()
plt.xlabel("Max. Profundidad")
plt.ylabel("RMSE")
plt.title('Curva de Validación - Árbol de Decisión')

In [ ]:
regresor = DecisionTreeRegressor(max_depth=10, random_state=42)
regresor.fit(X_train,y_train)

In [ ]:
modelo = ['Árbol de Decisión-PCA']

for i, model in enumerate([regresor]):
    y_train_pred = model.predict(X_train).reshape(50358,1)
    y_test_pred = model.predict(X_test).reshape(21582,1)
    
    print(f'Modelo: {modelo[i]}')
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print(f'Raíz del error cuadrático medio en Train: {rmse_train}')
    print(f'Raíz del error cuadrático medio en Test: {rmse_test}')
    
    plt.figure(figsize = (12,5))

    plt.subplot(1,2,1)
    sns.distplot(y_train - y_train_pred, bins = 25, label = 'train')
    sns.distplot(y_test - y_test_pred, bins = 25, label = 'test')
    plt.xlabel('errores')
    plt.title('Histograma de Errores')
    plt.legend()

    ax = plt.subplot(1,2,2)
    ax.scatter(y_test,y_test_pred, s =2)    
    lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  
    np.max([ax.get_xlim(), ax.get_ylim()]),  
    ]
    
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    plt.xlabel('y (test)')
    plt.ylabel('y_pred (test)')
    plt.title('Gráfico de Dispersión')
    
    plt.tight_layout()
    plt.show()

* *Los RMSE tanto para Train como para Test, resultan mucho menores a los obtenidos en el Proyecto 1.*

In [ ]:
regresor.feature_importances_

* *La primer característica tiene una mayor importancia relativa a la hora de realizar predicciones (su importancia es del 88%).*

## 4. Evaluación del desempeño del modelo obtenido, luego de transformar los datos.

* *Cabe aclarar, que en la presente transformación de datos, incluímos el escalado de los mismos, lo cual no se realizó en el Proyecto 01.*
* *Es por ello que:*
    * *Con el fin de **comparar bajo las mismas escalas**: se procederá a replicar los modelos realizados en el Proyecto 01, aunque realizando el escalado de sus datos.*
    * *Para sumar una comparación más: se contrastarán los valores originales obtenidos en el Proyecto 01 (sin escalado), con el del Presente Proyecto pero **sin escalar**, es decir, se aplicará el modelo de Árbol de Decisión a partir del dataset **properati_6** (habiendo **aplicado PCA**).*

### 4.1. Repetimos los modelos aplicados en el Proyecto 01, realizando el Escalado de los Datos

#### 4.1.1 Replicamos los Filtros del Primer Proyecto de Machine Learning (Proyecto 01) y Escalamos los Datos

In [ ]:
properati_ml = pd.read_csv('DS_Proyecto_01_Datos_Properati.csv')

In [ ]:
properati_ml_2 = properati_ml [(properati_ml['l2'] == 'Capital Federal') & ((properati_ml['property_type'] == 'Departamento') | (properati_ml['property_type'] == 'PH') | (properati_ml['property_type'] == 'Casa'))]

In [ ]:
properati_ml_3 = properati_ml_2 [(properati_ml_2['surface_total'] >= 15) & (properati_ml_2['surface_total'] <= 1000)]

In [ ]:
properati_ml_4 = properati_ml_3 [(properati_ml_3['price'] <= 4000000)]

In [ ]:
properati_ml_5 = properati_ml_4.loc[:, ['rooms','bedrooms','bathrooms','surface_total','surface_covered','price']]

In [ ]:
properati_ml_5.isnull().sum()

In [ ]:
properati_ml_5.dropna()

__Checkpoint:__ deberías obtener un dataset con 81019 instacias y 6 columnas.

In [ ]:
properati_ml_5.shape

* *Cabe aclarar, que el Proyecto 01 tiene un **Error**, ya que si bien el Checkpoint dió OK, no fue aplicado el último filtro en un nuevo dataframe, por lo que el total de instancias con las que se trabajó, fue de **82.373**.*
* *A fin de comparar, se utilizará dicho dataset para no modificar los resultados finales*.

In [ ]:
properati_scaler_ml = RobustScaler().fit_transform(properati_ml_5[['rooms','bedrooms','bathrooms','surface_total','surface_covered','price']])

properati_scaler_ml.shape

In [ ]:
properati_ml_5_sclaer = pd.DataFrame(properati_scaler_ml, index = properati_ml_5[['rooms','bedrooms','bathrooms','surface_total','surface_covered','price']].index,
                         columns = properati_ml_5[['rooms','bedrooms','bathrooms','surface_total','surface_covered','price']].columns)

#### 4.1.2 Regresión Lineal

In [ ]:
X = properati_ml_5_sclaer[['rooms','surface_total']]
y = properati_ml_5_sclaer[['price']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

In [ ]:
linear_model.fit(X_train, y_train)
print(linear_model.coef_, linear_model.intercept_) # Pendiente y ordenada al origen

* *Se realiza la evaluación del modelo.*
* *Se realiza el `histograma de los errores` ( y−y_predicho ) para cada conjunto y el `gráfico de dispersión` de  y  vs  y_predicho  para el conjunto de test.*

In [ ]:
modelo = ['Regresión lineal']

for i, model in enumerate([linear_model]):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    print(f'Modelo: {modelo[i]}')
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print(f'Raíz del error cuadrático medio en Train: {rmse_train}')
    print(f'Raíz del error cuadrático medio en Test: {rmse_test}')
    
    plt.figure(figsize = (12,5))

    plt.subplot(1,2,1)
    sns.distplot(y_train - y_train_pred, bins = 25, label = 'train')
    sns.distplot(y_test - y_test_pred, bins = 25, label = 'test')
    plt.xlabel('errores')
    plt.title('Histograma de Errores')
    plt.legend()

    ax = plt.subplot(1,2,2)
    ax.scatter(y_test,y_test_pred, s =2)    
    lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  
    np.max([ax.get_xlim(), ax.get_ylim()]),  
    ]
    
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    plt.xlabel('y (test)')
    plt.ylabel('y_pred (test)')
    plt.title('Gráfico de Dispersión')
    
    plt.tight_layout()
    plt.show()

* *Como se indicó en el proyecto anterior, si bien los histogramas de los errores parecieran ser algo simétricos, no se observan parecidos entre ambos conjuntos, siendo un indicador de que **no** nos estamos aproximando a nuestros datos de manera correcta.*
* *En el gráfico, **y**  vs  **y predicho**  para el conjunto de test, los puntos se van alejando de la diagonal, y el error, parece ser cada vez mayor.*
* *En éste caso, la regresión lineal no estaría haciendo un buen trabajo en reproducir la curva teórica.*

#### 4.1.3. Árboles de Decisión y K Vecinos Más Cercanos

In [ ]:
X = properati_ml_5_sclaer[['rooms','surface_total']]
y = properati_ml_5_sclaer[['price']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=10)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
tree_regressor = DecisionTreeRegressor(max_depth=25, random_state=10)
knn_regressor = KNeighborsRegressor(n_neighbors=8)

In [ ]:
tree_regressor.fit(X_train, y_train)
knn_regressor.fit(X_train, y_train)

In [ ]:
modelos = ['Árbol de Decisión', 'K Vecinos Más Cercanos']
for i, model in enumerate([tree_regressor, knn_regressor]):
    y_train_pred = model.predict(X_train).reshape(57661,1)
    y_test_pred = model.predict(X_test).reshape(24712,1)
    
    print(f'Modelo: {modelos[i]}')
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print(f'Raíz del error cuadrático medio en Train: {rmse_train}')
    print(f'Raíz del error cuadrático medio en Test: {rmse_test}')
    
    plt.figure(figsize = (12,5))

    plt.subplot(1,2,1)
    sns.distplot(y_train - y_train_pred, bins = 25, label = 'train')
    sns.distplot(y_test - y_test_pred, bins = 25, label = 'test')
    plt.xlabel('errores')
    plt.title('Histograma de Errores')
    plt.legend()

    ax = plt.subplot(1,2,2)
    ax.scatter(y_test,y_test_pred, s =2)    
    lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]
    
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    plt.xlabel('y (test)')
    plt.ylabel('y_pred (test)')
    plt.title('Gráfico de Dispersión')
    
    plt.tight_layout()
    plt.show()

* *Tanto con Árboles de Decisión como en K Vecinos, los histogramas de los errores no parecen ser simétricos ni parecidos para ambos conjuntos, siendo un indicador de que **no** nos estamos aproximando a nuestros datos de manera correcta.*
* *En los gráficos, **y**  vs  **y predicho**  para el conjunto de test, los puntos se encuentran cada vez más dispersos, y el error, es ser cada vez mayor.*

### 4.2. Modelo de Árbol de Decisión a partir del dataset **properati_6**

In [ ]:
X = properati_6[['surface_total','surface_total','bedrooms','rooms','bathrooms']]
y = properati_6[['price']]

features = X.T
cov_matrix = np.cov(features)

valores, vectores = np.linalg.eig (cov_matrix)

from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
print("original shape:   ", X.shape)
print("transformed shape:", X_pca.shape)

print(pca.components_)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.30, random_state=10)

In [ ]:
tree_regressor = DecisionTreeRegressor(max_depth=10, random_state=42)

In [ ]:
tree_regressor.fit(X_train, y_train)

In [ ]:
y_train_pred = tree_regressor.predict(X_train)
y_test_pred = tree_regressor.predict(X_test)

In [ ]:
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f'Raíz del error cuadrático medio en Train: {rmse_train}')
print(f'Raíz del error cuadrático medio en Test: {rmse_test}')

### 4.3. Evaluación del Desempeño

                                                     Modelos Con Escalado de Datos

|   Modelos Proyecto 01  | RMSE Train | RMSE Test | Hiperparámetros Utilizados |
|:----------------------:|:----------:|:---------:|:--------------------------:|
|    Regresión Lineal    |    1.377   |   1.443   |             --             |
|    Árbol de Decisión   |    0.955   |   1.259   |       max_depth = 25       |
| K Vecinos Más Cercanos |    1.131   |   1.265   |       n_neighbors = 8      |

|    Modelo Proyecto 02   | RMSE Train | RMSE Test | Hiperparámetro Utilizado |
|:-----------------------:|:----------:|:---------:|:------------------------:|
| Árbol de Decisión (PCA) |    0.458   |   0.470   |      max_depth = 10      |

                                                     Modelos Sin Escalado de Datos

|   Modelos Proyecto 01  | RMSE Train | RMSE Test | Hiperparámetros Utilizados |
|:----------------------:|:----------:|:---------:|:--------------------------:|
|    Regresión Lineal    |  213500.2  |  223733.9 |             --             |
|    Árbol de Decisión   |  148049.8  |  195382.0 |       max_depth = 25       |
| K Vecinos Más Cercanos |  174201.2  |  197995.1 |       n_neighbors = 8      |

|    Modelo Proyecto 02   | RMSE Train | RMSE Test | Hiperparámetro Utilizado |
|:-----------------------:|:----------:|:---------:|:------------------------:|
| Árbol de Decisión (PCA) |   54696.3  |  56087.7  |      max_depth = 10      |

* *Según se expone, tanto en el caso de modelos escalados como no escalados, **el Desempeño del Modelo Obtenido del Proyecto 02, es mucho mejor** que los extrtraídos del proyecto anterior.*

* *En los 3 modelos aplicados en el Proyecto 01, los errores RMSE, tienen valores muy altos, por lo que no logran una buena predicción de los precios.*
* *Ésto ocurre, porque los errores RMSE, son sensibles a valores atípicos, y en la muestra del Proyecto 01, tenemos varios, ya que en su oportunidad, no se realizó el filtrado de datos más adecuado para reducir los Outliers de la mejor manera posible, ni tampoco imputación de datos o reducción de dimensionalidad*.
* *Además, hay un notable sobreajuste en los modelos ejecutados en el Proyecto 01, visualizado a través de la importante diferencia existente entre el RMSE del Train y del Test (el primero mucho menor).*

## PARTE B - Modelos Avanzados ##

* Elige dos de los modelos avanzados vistos (en el caso de regresión, considera una regresión lineal con atributos polinómicos y regularización). Entrénalos y evalúalos con sus argumentos por defecto. No te olvides de hacer un train/test split y usar Validación Cruzada.

* Optimiza sus hiperparámetros mediante Validación Cruzada y Grid Search o Random Search.

* Compara el desempeño de los nuevos modelos entre sí y con el modelo de la Parte A. ¿Cuál elegirías? Justifica.

* *__Aclaración:__ Los clasificadores basados en modelos gráficos, como Fisher LDA o Naive Bayes, así como los Árboles de decisión y los métodos de conjuntos basados en Árboles (RF, XGB) son invariantes al escalado de características (no lo requieren para converger a los errores mínimos), pero aún así, podría ser una buena idea reescalar / estandarizar su datos.  
Es por ello, que en el presente apartado, se optó por elegir el dataset **properati_7**, el cual incluye el escalado de los datos.*

### 1. Regresión Lineal con Atributos Polinómicos (Polynomial Features) y Regularización

#### 1.1 Regresión Lineal con Atributos Polinómicos

* *La regresión polinomial es otra forma de regresión en la que la potencia máxima de la variable independiente es más de 1. En esta técnica de regresión, la línea de mejor ajuste no es una línea recta, sino que tiene la forma de una curva.*

In [ ]:
X = properati_7[['bedrooms','surface_total','surface_covered']]
y = properati_7[['price']]

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=10)

In [ ]:
def evaluar_regresion(model,X,y, X_train, X_test, y_train, y_test):
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculamos el Error
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

    print(f'Raíz del error cuadrático medio en Train: {rmse_train}')
    print(f'Raíz del error cuadrático medio en Test: {rmse_test}')

    plt.figure(figsize = (17,5))
    
    plt.subplot(1,3,2)
    sns.distplot(y_train - y_train_pred, bins = 20, label = 'train')
    sns.distplot(y_test - y_test_pred, bins = 20, label = 'test')
    plt.xlabel('errores')
    plt.legend()

    ax = plt.subplot(1,3,3)
    ax.scatter(y_test,y_test_pred, s =2)

    lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes]
    ]

    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    plt.xlabel('y (test)')
    plt.ylabel('y_pred (test)')

    plt.tight_layout()
    plt.show()

In [ ]:
# Visualizamos cuál es el mejor Grado (Degree) a aplicar
rmses = []
degrees = np.arange(1, 10)
min_rmse, min_deg = 1e10, 0

for deg in degrees:

    poly_features = PolynomialFeatures(degree=deg, include_bias=False)
    X_poly_train = poly_features.fit_transform(X_train)

    poly_reg = LinearRegression()
    poly_reg.fit(X_poly_train, y_train)

    X_poly_test = poly_features.fit_transform(X_test)
    poly_predict = poly_reg.predict(X_poly_test)
    poly_mse = mean_squared_error(y_test, poly_predict)
    poly_rmse = np.sqrt(poly_mse)
    rmses.append(poly_rmse)
    
    if min_rmse > poly_rmse:
        min_rmse = poly_rmse
        min_deg = deg

print('Best degree {} with RMSE {}'.format(min_deg, min_rmse))
        
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(degrees, rmses)
ax.set_yscale('log')
ax.set_xlabel('Degree')
ax.set_ylabel('RMSE')
plt.title('Degree Vs. RMSE')

* *Si bien el gráfico expone un punto mínimo en Grado = 5, parece ser una línea bastante recta desde Grado = 3 hasta dicho punto.*
* *Por ello, y dado que nuestro dataset tiene varios atributos, el modelo se llevará a cabo con un polinomio de grado 3.*

In [ ]:
poly = PolynomialFeatures (3, include_bias = False) # Preprocesamiento, de 'Grado 3'
X_train_new = poly.fit_transform(X_train) # Conjunto de datos que incluye los nuevos atributos, las nueva columnas
X_test_new = poly.fit_transform(X_test)

# Ahora son 3 columnas. Son nuevas combinaciones lineales del atributo original.
# A la vista, elevó a la 1, a la 2, a la 3

In [ ]:
reg = LinearRegression()
reg.fit(X_train_new, y_train)
print(f'Pendientes: {reg.coef_}')
print(f'Ordenada: {reg.intercept_}')

In [ ]:
y_train_pred_reg = reg.predict(X_train_new)
y_test_pred_reg = reg.predict(X_test_new)

In [ ]:
evaluar_regresion(reg, X,y, X_train_new, X_test_new, y_train, y_test)

#### 1.2 Regularización

* *En la regularización, lo que hacemos es mantener el mismo número de características, pero reducir la magnitud de los coeficientes.*

* *__Regulación L2 o RIDGE:__ Se agrega a la función de Costo, un término proporcional al cuadrado del valor de los coeficientes de peso. La penalización, encoge los coeficientes hacia el Cero (pero no llegan al cero absoluto).*
    * *El valor de Alpha, puede ser de 0.1 hasta el valor que se desee.*
    * *Cuanto mayor sea el valor de alfa, menos varianza exhibirá su modelo y menor será la dispersión de los datos.*
    * *Funciona bien si hay muchos parámetros grandes de aproximadamente el mismo valor.*   

* *__Regulación L1 o LASSO:__ Se agrega a la función de Costo, un término proporcional al valor absoluto de los coeficientes de peso. La penalización, encoge los coeficientes hacia el Cero o convierte a algunos coeficientes en Cero. Así, elimina las características menos importantes en nuestro modelo.*
    * *El valor de Alpha, puede variar de 0.1 a 1.*
    * *Tiende a funcionar bien si hay una pequeña cantidad de parámetros significativos y los otros están cerca de cero.* 

##### 1.2.1 Se entrena el modelo **Ridge**

In [ ]:
from sklearn.linear_model import Ridge, Lasso

reg_ridge = Ridge() # No se incluye alpha, ya que en la consigna se indica entrenar y evaluar con los argumentos por defecto
reg_ridge.fit(X_train_new,y_train)

print(f'Pendientes: {reg_ridge.coef_}')
print(f'Ordenada: {reg_ridge.intercept_}')

In [ ]:
evaluar_regresion(reg_ridge, X,y, X_train_new, X_test_new, y_train, y_test)

##### 1.2.2 Se entrena el modelo **Lasso**

In [ ]:
reg_lasso = Lasso()
reg_lasso.fit(X_train_new,y_train)

print(reg_lasso.coef_, reg_lasso.intercept_)

In [ ]:
def evaluar_regresion_2(model,X,y, X_train, X_test, y_train, y_test):
    
    y_train_pred = model.predict(X_train).reshape(50358,1)
    y_test_pred = model.predict(X_test).reshape(21582,1)
    
    # Calculamos el Error
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

    print(f'Raíz del error cuadrático medio en Train: {rmse_train}')
    print(f'Raíz del error cuadrático medio en Test: {rmse_test}')

    ### Graficamos los Resultados
    plt.figure(figsize = (17,5))
    
    plt.subplot(1,3,2)
    sns.distplot(y_train - y_train_pred, bins = 20, label = 'train')
    sns.distplot(y_test - y_test_pred, bins = 20, label = 'test')
    plt.xlabel('errores')
    plt.legend()

    ax = plt.subplot(1,3,3)
    ax.scatter(y_test,y_test_pred, s =2)

    lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes]
    ]

    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    plt.xlabel('y (test)')
    plt.ylabel('y_pred (test)')

    plt.tight_layout()
    plt.show()
    
evaluar_regresion_2(reg_lasso, X,y, X_train_new, X_test_new, y_train, y_test)

* *La regularización __`Ridge` resultó ser más efectiva__ que Lasso, y ésto es porque la mayoría de los predictores elegidos influyen en la respuesta (superficie total y cubierta influyen sobre la determinación de los precios).*
* *El modelo con regularización __`Ridge`__, se observa una distribución simétrica y más si la comparamos con Lasso.* 
* *Además, a simple vista se observa que para el caso de Lasso, en el gráfico **y**  vs  **y predicho**  para el conjunto de test, los puntos se van alejando de la diagonal, y el error, parece ser cada vez mayor.*

#### 1.3 Optimización de Hiperparámetros: RandomSerch

##### 1.3.1 Se determina el mejor `alpha` para el modelo Ridge, por ser el que mejor resultado arrojó

In [ ]:
#import scipy as sp

from sklearn.model_selection import RandomizedSearchCV # Búsqueda aleatoria en hiperparámetros.

In [ ]:
ridge = Ridge()

In [ ]:
# find optimal alpha with grid search
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_ramdom = dict(alpha=alpha)
ramdom = RandomizedSearchCV(estimator=ridge, param_distributions=param_ramdom, verbose=1, n_jobs=-1, n_iter = 100)
ramdom_result = ramdom.fit(X_train_new, y_train)
print('Best Score: ', ramdom_result.best_score_)
print('Best Params: ', ramdom_result.best_params_)

##### 1.3.2 Se entrena el modelo Ridge con el `Best Params` según RandomSearch

* *El resultado obtenido, debería ser el mismo que en el modelo Ridge anteriormente entrenado y evaluado, ya que el alpha por default es 1.0, que es el resultante en éste caso como Best Params.*

In [ ]:
reg_ridge_random = Ridge(alpha= 0.001)
reg_ridge_random.fit(X_train_new,y_train)

print(f'Pendientes: {reg_ridge.coef_}')
print(f'Ordenada: {reg_ridge.intercept_}')

In [ ]:
y_train_pred_random = reg_ridge_random.predict(X_train_new)
y_test_pred_random = reg_ridge_random.predict(X_test_new)

In [ ]:
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred_random))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred_random))
print(f'Raíz del error cuadrático medio en Train: {rmse_train}')
print(f'Raíz del error cuadrático medio en Test: {rmse_test}')

* *Se confirma que los RMSE obtenidos son los mismos.*
* *Para el caso de Polynomial Features, el similar el resultado del modelo antes y después de efectuada la Regularización.*

### 2. XG-Boost

In [ ]:
X = properati_7[['bedrooms','surface_total','surface_covered']]
y = properati_7[['price']]

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import GridSearchCV

#### 2.1 Se entrena y evalúa el modelo, con sus argumentos por defecto

In [ ]:
# Separamos los datos en train y test (held-out)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:
xgb = xgb.XGBRegressor() # Default n_estimators=100, max_depth=6

In [ ]:
xgb.fit(X_train, y_train, eval_metric=['error'])

In [ ]:
y_train_pred_xgb = xgb.predict(X_train)
y_test_pred_xgb = xgb.predict(X_test)

In [ ]:
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred_xgb))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred_xgb))
print(f'Raíz del error cuadrático medio en Train: {rmse_train}')
print(f'Raíz del error cuadrático medio en Test: {rmse_test}')

In [ ]:
print(xgb.feature_importances_)

* *`Surface Total` es el atributo mas importante, el que mejor separa los datos.*

#### 2.2 Optimización de Hiperparámetros: GridSerch

In [ ]:
model = XGBRegressor()
n_estimators = [50,100,150,200,250,300]
max_depth = [2,4,6,8]

param_grid = dict(max_depth=max_depth, n_estimators=n_estimators)
kfold = KFold(n_splits=2, shuffle=True, random_state=0)
grid_search = GridSearchCV(model, param_grid=param_grid, n_jobs=-1, cv=kfold, verbose=1)
grid_result = grid_search.fit(X_train, y_train)

In [ ]:
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

#### 2.3 Se entrena el modelo XGB con los argumentos obtenidos de Gridsearch

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor

In [ ]:
xgb_grid = xgb.XGBRegressor(n_estimators=100, max_depth=8, random_state=10)

In [ ]:
xgb_grid.fit(X_train, y_train, eval_metric=['error'])

In [ ]:
modelo = ['XG-Boost']

for i, model in enumerate([xgb_grid]):
    y_train_pred = model.predict(X_train).reshape(50358,1)
    y_test_pred = model.predict(X_test).reshape(21582,1)
    
    print(f'Modelo: {modelo[i]}')
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print(f'Raíz del error cuadrático medio en Train: {rmse_train}')
    print(f'Raíz del error cuadrático medio en Test: {rmse_test}')
    
    plt.figure(figsize = (12,5))

    plt.subplot(1,2,1)
    sns.distplot(y_train - y_train_pred, bins = 25, label = 'train')
    sns.distplot(y_test - y_test_pred, bins = 25, label = 'test')
    plt.xlabel('errores')
    plt.title('Histograma de Errores')
    plt.legend()

    ax = plt.subplot(1,2,2)
    ax.scatter(y_test,y_test_pred, s =2)    
    lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  
    np.max([ax.get_xlim(), ax.get_ylim()]),  
    ]
    
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    plt.xlabel('y (test)')
    plt.ylabel('y_pred (test)')
    plt.title('Gráfico de Dispersión')
    
    plt.tight_layout()
    plt.show()

In [ ]:
print(xgb_grid.feature_importances_)

* *Si bien `Surface Total` sigue siendo el atributo que mejor separa los datos, `Surface Total` ha adquirido importancia mayor importancia.*
* *Habiendo definido los Hiperparámetros, el Error de Train mejoró (se redujo en 0.03), y algo menos el Error de Test (se redujo en 0.01).*

### 3. Comparación del desempeño de los nuevos modelos entre sí y con el modelo de la Parte A

|      Modelo Parte A     | RMSE Train | RMSE Test | Hiperparámetro Utilizado |
|:-----------------------:|:----------:|:---------:|:------------------------:|
| Árbol de Decisión (PCA) |    0.458   |   0.470   |      max_depth = 10      |


|     Modelos Parte B    | RMSE Train | RMSE Test |   Hiperparámetros Utilizados  |
|:----------------------:|:----------:|:---------:|:-----------------------------:|
|   Polynomial Features  |    0.480   |   0.480   |  alpha= 1.0, PolyFeatures = 3 |
|        XG-Boost        |    0.375   |   0.424   | n_estimators=100, max_depth=8 |

* *En éste caso, fue **XG-Boost fue el modelo con mejor desempeño**.*
* *Éste resultado, era realmente el esperado, ya que se trata de un algoritmo eficiente y fácil de usar que ofrece un alto rendimiento y precisión en comparación con otros algoritmos.*
* *Además, XGBoost tiene una regularización incorporada L1 (Regresión de lazo) y L2 (Regresión de cresta) que evita que el modelo se sobreajuste.*
* *XGBoost, por otro lado, hace divisiones hasta el max_depth especificado y luego comienza a podar el árbol hacia atrás y elimine las divisiones más allá de las cuales no hay ganancia positiva.*
* *Es por ello que se considera uno de los modelos más eficientes a la hora de realizar predicciones.*

* *El modelo de **Árbol de Decisión habiendo aplicado PCA**, posee igualmente, una buena capacidad de predicción, por el hecho de haber realizado reducción de dimensionalidad, pasando de 5 features a 2 estimadores.*
* *No se observa en éste caso Sobrejuste, ya que los RMSE obtenidos para Train y Test son similares, existiendo un equilibrio entre sesgo y varianza.*

* *En el caso de **Polynomial Features**, lo que se hizo fue crear muchos atributos nuevos con combinaciones distintas, y esperar aproximarse con esas combinaciones, a la función objetivo.*
* *El modelo se desarrolló con PolynomialFeatures de Grado 3, ya que si bien mayores grados podría haber dado lugar a un mejor desempeño, no era muy recomendable dado el número de features en nuestro dataset, y además porque su desempeño no parecía variar entre dicho Grado y el indicado como Óptimo (Grado = 5).*
* *No se observó sobreajuste en el modelo, ya que se llevó adelante la regularización a través de L2 Ridge, sin embargo no se obtuvieron tan buenos resultados como en el caso de Arboles de Decisión previo PCA*.

## PARTE C - Interpretación de modelos ##

De acuerdo a lo que el modelo permite, responde algunas o todas las siguientes preguntas:

* ¿Qué variables fueron relevantes para el modelo para hacer una predicción? ¿Cuáles no? Si usaste una regresión lineal con regularización, presta atención a los parámetros (pendientes) obtenidas. Si usaste un modelo de ensamble en árboles, además de ver la importancia de cada atributo, también elige algunos árboles al azar y observa qué atributos considera importantes. ¿En qué se diferencian esos árboles? ¿Por qué? Finalmente, responde, ¿coincide con lo que esperabas a partir de tu experiencia con este dataset?

* ¿Cómo es la distribución de errores (regresión) o qué clases se confunden entre sí (clasificación)? ¿Dónde falla? ¿A qué se debe?

#### Variables:
* *La variable más relevante a la hora de hacer predicciones, fue `Superficie Total`, y en algunos casos, como segunda más importante, `Superficie Cubierta`.*
* *Las variables menos importantes y por ello no utilizadas para la mayoría de los modelos, fueron `bathrooms` y `rooms`.*
* *Cabe aclarar que, antes de determinar cuáles fueron las variables que no se utilizarían en los modelos, se realizaron pruebas con las mismas.*

#### Pendiente Obtenida en Regresión Polinómica con Regularización:
* *El target es el valor que desea predecir (y = precio). Entonces, como la Regresión Ridge (elegida) puede predecir más valores para cada instancia (no solo uno), lo que va a mostrar **coef_** es los coeficientes para la predicción de cada uno de dichos targets.*
* *En este caso, el coeficiente es la pendiente de la línea ajustada y la intersección es el punto donde la línea ajustada se cruza con el eje y.*
* *Como se observa, en el gráfico **y**  vs  **y predicho**  para el conjunto de test, los puntos si bien comienzan desde una distancia relativa de la diagonal (pendiente), se van alejando algo más de la misma, y el error, parece ser cada vez mayor.*
* *Igualmente su desempeño es notablemente mejor que para el caso de Regresión Lineal, que comienza con sus puntos coincidentes respecto a la pendiente y luego se separan progresivamente.*

#### Distribución de los Errores:
* *Es en el gráfico de error de XG-Boost, donde se visualiza una mayor robustez en los datos.*
* *Asimismo, en el gráfico, **y**  vs  **y predicho** para el conjunto de test, para el mismo modelo, si bien los puntos se van alejando de la diagonal, el error parece mantenerse en los mismos rangos relativos.*

## DESAFÍO OPCIONAL ##

Aplica una técnica de Clustering sobre el dataset. Puedes combinar con técnicas de reducción de dimensionalidad para facilitar la visualización. ¿Qué clusters encuentras? ¿A qué pueden corresponder? Te dejamos preguntas que pueden servir como disparadoras: ¿qué barrios se parecen más entre sí?¿qué tipos de propiedades se parecen más entre sí?

## Clustering

### 1. K-Means

In [ ]:
from sklearn import preprocessing
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.datasets import make_blobs, make_moons

* *Se utiliza el dataset **properati_6**, ya que incluye el feature `l3`, que es sobre el cual se trabajará para determinar los Clusters existentes.*

1.1 Se aplica **Dummies** sobre la variable `Barrios`.

In [ ]:
dummies = pd.get_dummies(properati_6['l3'])
dummies.head()

* *Dado que ha sido creado un marco de datos completamente nuevo, y para compararlo con el marco de datos original, es necesario fusionarlos o concatenarlos para que funcionen correctamente. Al crear variables ficticias, se crean nuevas columnas para el conjunto de datos original.*

* *Se coloca la variable ficticia en el lado derecho del marco de datos.*

In [ ]:
barrios = pd.concat([properati_6,dummies], axis = 1)
barrios.head()

In [ ]:
X = barrios[['l3']]
y = barrios[['price']]

In [ ]:
print (X)

1.2 Se determina el **número de Centros**.

In [ ]:
n_samples = 71940
n_centros = 8
X, y = make_blobs(centers=n_centros, cluster_std=0.3, n_features=2, random_state=0)

* *Se grafican los datasets.*

In [ ]:
sns.scatterplot(x = X[:,0], y = X[:,1], hue = y, legend ='full')
plt.show()

* *Se evaluarán las particiones a realizar mediante KMeans usando la distancia al centroide. La idea es que al variar el número de clúster K en el modelo, el valor de la distancia media de los datos al centroide más cercano va a variar.*
* *Se grafica dicha curva, para elegir el número de particiones óptimos con el metodo del codo.*
* *Se realiza una lista con las distancias medias a los centroides en el dataset 1, probando con un número de clústers que va de 2 a 14.*

In [ ]:
lista_distancias_medias = []

K = np.arange(2,14)
for k in K:
   
    km = KMeans(n_clusters=k)
    km = km.fit(X)
   
    distancia_total = km.inertia_
    # Inercia: la suma de la distancia al cuadrado de cada punto con su respectivo centroide
    
    distancia_media = distancia_total / n_samples
    lista_distancias_medias.append(distancia_media)
    # Distorsión: el promedio de todas las distancias de los centroides con sus respectivos puntos al cuadrado.

* *Se grafica la distancia media en función del número de clústers.*

In [ ]:
plt.figure(figsize = (10,7))
# Graficamos una linea continua y tambien unos puntos para resaltar los valores enteros de K.
plt.plot(K, lista_distancias_medias, lw=3)
plt.scatter(K, lista_distancias_medias, s=55,c='r')
plt.xlabel('Cantidad de Clusters K')
plt.ylabel('Inercia media')
plt.title('Método del Codo')

plt.show()

* *S/ el gráfico, parecería que el n_clusters más acorde sería de 6.*
* *Se busca el mejor numero de k en cada caso según la curva y se grafican los clusters para cada dataset.*

In [ ]:
# Defino y entreno el modelo
km = KMeans(n_clusters=6)
km = km.fit(X)

# Obtengo la posición de los centros y las etiquetas
etiquetas_ = km.labels_
centros_ = km.cluster_centers_

# Graficamos los centros de los clusters y los datapoints
sns.scatterplot(X[:, 0], X[:, -1], style = etiquetas_)
sns.scatterplot(centros_[:, 0], centros_[:, 1],color='black', marker="+", s=1000)
plt.title('Data points y Centroides de los Clusters')
plt.show()

### 2. Silhouette

* *A continuación, se evalúan las particiones mediante el valor de silhouette.*
* *Al variar los parámetros de los modelos de clustering, cambiará la distribución del valor de Silhouettes en los datos. Con esa distribución, se elegirán los mejores parametros posibles (cohesión y separación).*

*__Nota 1:__ el coeficiente de Silhoutte va de -1 a 1.*

*__Nota 2:__ el Silhouette promedio también va del -1 al 1, donde en 1 los clusters están bien separados, en 0 están cerca y en -1 el los clusters están mezclados. Esto nos da una idea de cuán buena es la separación en clusters.*

* *En ésta oportunidad, vamos a calcular el valor de silhouette usando la función `silhouette_score`.*

In [ ]:
# Preparamos una lista donde vamos a ir agregando los valores medios de silhouette
lista_sil = []
# Fiteammos un modelo para cada numero de cluster que queremos testear
for k in range(2,14):
    # Definimos y entrenamos el modelo
    km = KMeans(n_clusters=k)
    km = km.fit(X)
    
    # Tomamos las etiquetas
    etiquetas = km.labels_
    
    # Calculamos el silhouette 
    valor_medio_sil = silhouette_score(X, etiquetas)
    lista_sil.append(valor_medio_sil)
    
plt.figure(figsize = (10,7))
plt.plot(K, lista_sil, lw=3)
plt.scatter(K, lista_sil,s=55,c='r')
plt.xlabel('Cantidad de Clusters K')
plt.ylabel('Silhouette Media')
plt.title('Silhouette Media por Nro. de Clusters')
plt.show()

* *El gráfico, representa el Promedio de todas las siluetas por cantidad de clusters.*
* *Si contrastamos con la próxima gráfica de silueta, que se verá de costado, vendría a ser la línea punteada observada.*

In [ ]:
X_std = X

fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_size_inches(18, 7)
    
km = KMeans(n_clusters=8)
labels = km.fit_predict(X_std)
centroids = km.cluster_centers_

silhouette_vals = silhouette_samples(X_std, labels)

y_ticks = []
y_lower, y_upper = 0, 0
for i, cluster in enumerate(np.unique(labels)):
    cluster_silhouette_vals = silhouette_vals[labels == cluster]
    cluster_silhouette_vals.sort()
    y_upper += len(cluster_silhouette_vals)
    ax1.barh(range(y_lower, y_upper), cluster_silhouette_vals, edgecolor='none', height=1)
    ax1.text(-0.03, (y_lower + y_upper) / 2, str(i + 1))
    y_lower += len(cluster_silhouette_vals)

avg_score = np.mean(silhouette_vals)
ax1.axvline(avg_score, linestyle='--', linewidth=2, color='green')
ax1.set_yticks([])
ax1.set_xlim([-0.1, 1])
ax1.set_xlabel('Valores del Coeficiente de Silueta')
ax1.set_ylabel('Clusters')
ax1.set_title('Diagrama de Siluetas', y=1.02);

# Scatter plot of data colored with labels
ax2.scatter(X_std[:, 0], X_std[:, 1], c=labels)
ax2.scatter(centroids[:, 0], centroids[:, 1], marker='*', c='r', s=250)
ax2.set_xlim([-12, 12])
ax2.set_xlim([-12, 12])
ax2.set_xlabel('Tiempo de Emisión en Minutos')
ax2.set_ylabel('Tiempo de Espera p/ Próxima Emisión')
ax2.set_title('Visualización de los Clusters', y=1.02)
ax2.set_aspect('equal')
plt.tight_layout()
plt.suptitle(f'Análisis de Silueta utilizando k = {6}', fontsize=16, fontweight='semibold', y=1.05);

* *Para entender mejor el gráfico:*
    * *En el eje x está el valor de Silhouette, en el eje y los clusters.*
    * *El valor de silhouette de cada instancia está graficado como una barrita muy finita. Están todas las instancias graficadas, ordenadas de mayor a menor.*
    * *la línea punteada roja representa el Silhouette promedio de toda la partición.*

* *Como se observa, el mejor número de Clusters para separar los datos es **k=8**.*
* *Sin embargo, si bien sabemos que los barrios se encuentran dividimos en 6 grupos, no tenemos certeza de cuales de ellos pertenecen a cada grupo.*

### 3. DBSCAN

1. Se importan los módulos de Python necesarios.

In [ ]:
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

2. Se eliminan las coordenadas nulas.

In [ ]:
barrios.isnull().sum()

In [ ]:
df = barrios.dropna() # Se eliminan todos los núlos porque sólo se trata de lat y lon (variables que vamos a utilizar)
df.shape

3. Se convierten las columnas de coordenadas de latitud y longitud en una matriz numérica bidimensional.

In [ ]:
coords = df[['lat', 'lon']].to_numpy()

4. Se **calcula DBSCAN**.

* *Se utiliza la métrica de Haversine y el algoritmo de Ball Tree, para calcular distancias de grandes círculos entre puntos.*
* *El parámetro **épsilon** es la distancia máxima (0,5 km en este ejemplo, por tratarse de barrios) que los puntos pueden estar entre sí para ser considerados un grupo.*
* *El parámetro **min_samples** es el tamaño mínimo del clúster (todo lo demás se clasifica como ruido). Al ser min_samples=1, cada punto de datos será asignado a un grupo.*
* *A diferencia de k-means, DBSCAN no requiere que especifique el número de clústeres por adelantado; los determina automáticamente en función de los parámetros épsilon y min_samples.*

In [ ]:
kms_per_radian = 6371.0088
epsilon = 0.5 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))

In [ ]:
import matplotlib.cm as cmx
import matplotlib.colors as colors

# define a helper function to get the colors for different clusters
def get_cmap(N):
    '''
    Returns a function that maps each index in 0, 1, ... N-1 to a distinct 
    RGB color.
    '''
    color_norm  = colors.Normalize(vmin=0, vmax=N-1)
    scalar_map = cmx.ScalarMappable(norm=color_norm, cmap='nipy_spectral') 
    def map_index_to_rgb_color(index):
        return scalar_map.to_rgba(index)
    return map_index_to_rgb_color

In [ ]:
from mpl_toolkits.basemap import Basemap

In [ ]:
plt.figure(figsize = (12, 12))
m = Basemap(projection='merc', resolution='l', epsg = 4269, 
        llcrnrlon=-122.7,llcrnrlat=36.2, urcrnrlon=-120.8,urcrnrlat=37.5)

unique_label = np.unique(cluster_labels)

# get different color for different cluster
cmaps = get_cmap(n_clusters)

# plot different clusters on map, note that the black dots are 
# outliers that not belone to any cluster. 
for i, cluster in enumerate(clusters):
    lons_select = cluster[:, 1]
    lats_select = cluster[:, 0]
    x, y = m(lons_select, lats_select)
    m.scatter(x,y,5,marker='o',color=cmaps(i), zorder = 10)

m.arcgisimage(service='World_Shaded_Relief', xpixels = 5000, verbose= False)

plt.show()

* *Como vemos, coincide con el número de Clusters arrojados por Silhouette.*

5. Se busca encontrar el **punto más central** de un grupo.

In [ ]:
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)
centermost_points = clusters.map(get_centermost_point)

In [ ]:
lats, lons = zip(*centermost_points)
rep_points = pd.DataFrame({'lon':lons, 'lat':lats})

In [ ]:
rs = rep_points.apply(lambda row : df[(df['lat']==row['lat']) & (df['lon']==row['lon'])].iloc[0], axis=1)

In [ ]:
from mpl_toolkits.basemap import Basemap

In [ ]:
# Plot the location of the earthquakes
plt.figure(figsize = (12, 12))

m = Basemap(projection='merc', resolution='l', epsg = 4269, 
            llcrnrlon=-122.7,llcrnrlat=36.2, urcrnrlon=-120.8,urcrnrlat=37.5)

# plot the aftershock
x, y = m(coords[:, 1], coords[:, 0])
m.scatter(x,y,5,marker='o',color='b')
m.arcgisimage(service='World_Shaded_Relief', xpixels = 5000, verbose= False)
    
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=[10, 6])
rs_scatter = ax.scatter(rs['lon'], rs['lat'], c='#99cc99', edgecolor='None', alpha=0.7, s=120)
df_scatter = ax.scatter(df['lon'], df['lat'], c='k', alpha=0.9, s=3)
ax.set_title('Full data set vs DBSCAN reduced set')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend([df_scatter, rs_scatter], ['Full set', 'Reduced set'], loc='upper right')
plt.show()